In [1]:
from __future__ import division
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
pd.options.display.max_rows = 12
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 4))

## 10장 실습
### 10.1. 주간 평균 수익률 구하기
1. stock_px.csv 파일을 시계열 데이터 close_px 로 읽어라.
 - 날짜 데이터를 datetime으로 읽어야 한다.
 - 첫번재 컬럼을 index로 지정해야 한다.
2. shift 메소드를 이용하여 수익율을 계산하여 rate_px를 생성한다.
3. resample을 이용하여 주간 Resampler 를 wSampler를 생성한다.
4. wSampler를 이용하여 주간 평균 수익률을 계산한다.
5. 2000년 한 해 동안 주간 평균 수익률이 가장 높은 종목과 그 주간은?

In [ ]:
path = 'C:/Users/tmznq/workspace/ml_scratch/seoul_coding_academy/pydata-book-1st-edition/ch11/stock_px.csv'

#### 1.1. 첫번째 컬럼을 index로 지정.

In [53]:
# close_px 데이터로 만들기
# index_col, parse_dates
close_px = pd.read_csv(path, index_col='Unnamed: 0', parse_dates=True)
close_px.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,4.98,7.86,2.87,16.79,4.27,0.51,6.04,328.79,6.12
1990-02-02,5.04,8.00,2.87,16.89,4.37,0.51,6.09,330.92,6.24
1990-02-05,5.07,8.18,2.87,17.32,4.34,0.51,6.05,331.85,6.25
1990-02-06,5.01,8.12,2.88,17.56,4.32,0.51,6.15,329.66,6.23
1990-02-07,5.04,7.77,2.91,17.93,4.38,0.51,6.17,333.75,6.33


In [54]:
close_px.index # dtype = 'object'

DatetimeIndex(['1990-02-01', '1990-02-02', '1990-02-05', '1990-02-06',
               '1990-02-07', '1990-02-08', '1990-02-09', '1990-02-12',
               '1990-02-13', '1990-02-14',
               ...
               '2011-10-03', '2011-10-04', '2011-10-05', '2011-10-06',
               '2011-10-07', '2011-10-10', '2011-10-11', '2011-10-12',
               '2011-10-13', '2011-10-14'],
              dtype='datetime64[ns]', length=5472, freq=None)

In [10]:
# from dateutil.parser import parse
# parse(data.index)

#### 1.2. index를 datetime으로 변환

In [22]:
close_px.index = pd.to_datetime(close_px.index)

In [96]:
close_px['2010']

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-01-04,16.40,214.01,14.66,128.32,60.84,29.76,58.10,1132.99,66.16
2010-01-05,15.89,214.38,14.74,126.77,60.14,29.77,58.80,1136.52,66.42
2010-01-06,16.72,210.97,14.66,125.94,60.63,29.58,58.21,1137.14,66.99
2010-01-07,16.36,210.58,15.42,125.51,60.19,29.28,57.84,1141.69,66.78
2010-01-08,16.77,211.98,15.75,126.77,60.40,29.48,57.65,1144.98,66.52
2010-01-11,17.19,210.11,15.90,125.44,60.41,29.10,57.59,1146.98,67.26
...,...,...,...,...,...,...,...,...,...
2010-12-23,15.25,323.60,17.60,144.04,60.62,27.78,64.20,1256.77,71.93
2010-12-27,15.14,324.68,17.75,143.50,60.31,27.55,63.94,1257.54,71.74
2010-12-28,15.16,325.47,17.87,143.86,60.43,27.49,63.80,1258.51,72.15


- 바뀐 거 확인

#### 2. shift 함수를 이용해서 수익률을 계산하여 rate_px 생성

In [67]:
rate_px = (close_px / close_px.shift(1) - 1).dropna()
rate_px.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-02,0.012048,0.017812,0.000000,0.005956,0.023419,0.0,0.008278,0.006478,0.019608
1990-02-05,0.005952,0.022500,0.000000,0.025459,-0.006865,0.0,-0.006568,0.002810,0.001603
1990-02-06,-0.011834,-0.007335,0.003484,0.013857,-0.004608,0.0,0.016529,-0.006599,-0.003200
1990-02-07,0.005988,-0.043103,0.010417,0.021071,0.013889,0.0,0.003252,0.012407,0.016051
1990-02-08,0.000000,-0.007722,0.003436,-0.003904,0.018265,0.0,0.008104,-0.002367,0.003160


#### 3. wsampler 생성 

In [68]:
wsampler = rate_px.resample('W')
wsampler

DatetimeIndexResampler [freq=<Week: weekday=6>, axis=0, closed=right, label=right, convention=start, base=0]

#### 4. 주간 평균 수익률 계산

In [69]:
w_mean = wsampler.mean()
w_mean.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-04,0.012048,0.017812,0.000000,0.005956,0.023419,0.000000,0.008278,0.006478,0.019608
1990-02-11,0.000815,0.000391,0.004837,0.010849,0.005481,0.003922,0.004906,0.001647,0.004153
1990-02-18,0.001686,-0.002216,-0.004077,0.001964,-0.000827,0.003846,-0.004148,-0.000516,-0.001087
1990-02-25,-0.001440,-0.003662,-0.006104,-0.002007,-0.015451,0.000605,-0.009518,-0.006491,-0.004348
1990-03-04,0.013098,0.003184,0.007732,0.004351,0.010759,0.014820,0.000411,0.006939,-0.001459


#### 5. 2010년 한 해 동안 주간 평균 수익률이 가장 높은 종목과 그 주간은?

In [70]:
# 5.1. 가장 높은 종목

In [71]:
year_mean = w_mean.resample('Y').mean()
year_mean.loc['2010']

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-12-31,0.000035,0.001917,0.000923,0.000585,0.000054,-0.000214,0.000453,0.000512,0.000411


In [89]:
year_mean.loc['2010'].rank(axis = 1, method= 'first', ascending=False)

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-12-31,8.0,1.0,2.0,3.0,7.0,9.0,5.0,4.0,6.0


- 2010년의 평균 수익률은 AAPL이 가장 높다.

In [90]:
year_mean.loc['2010'].resample('W')

DatetimeIndexResampler [freq=<Week: weekday=6>, axis=0, closed=right, label=right, convention=start, base=0]

In [91]:
# 5.2. 주간 구하기

In [92]:
year_mean2 = w_mean.resample('Y')

In [93]:
# argmax()

___
#### 5. 2010년 한 해 동안 주간 평균 수익률이 가장 높은 종목과 그 주간은?

In [94]:
w_mean.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-04,0.012048,0.017812,0.000000,0.005956,0.023419,0.000000,0.008278,0.006478,0.019608
1990-02-11,0.000815,0.000391,0.004837,0.010849,0.005481,0.003922,0.004906,0.001647,0.004153
1990-02-18,0.001686,-0.002216,-0.004077,0.001964,-0.000827,0.003846,-0.004148,-0.000516,-0.001087
1990-02-25,-0.001440,-0.003662,-0.006104,-0.002007,-0.015451,0.000605,-0.009518,-0.006491,-0.004348
1990-03-04,0.013098,0.003184,0.007732,0.004351,0.010759,0.014820,0.000411,0.006939,-0.001459


In [97]:
y_mean = w_mean.resample('Y')
for i in y_mean:
    print(i)

(Timestamp('1990-12-31 00:00:00', freq='A-DEC'),                   AA      AAPL        GE       IBM       JNJ      MSFT  \
1990-02-04  0.012048  0.017812  0.000000  0.005956  0.023419  0.000000   
1990-02-11  0.000815  0.000391  0.004837  0.010849  0.005481  0.003922   
1990-02-18  0.001686 -0.002216 -0.004077  0.001964 -0.000827  0.003846   
1990-02-25 -0.001440 -0.003662 -0.006104 -0.002007 -0.015451  0.000605   
1990-03-04  0.013098  0.003184  0.007732  0.004351  0.010759  0.014820   
1990-03-11 -0.002971  0.017897  0.000716  0.003185  0.002326  0.003634   
...              ...       ...       ...       ...       ...       ...   
1990-11-25  0.005649  0.008846 -0.007529 -0.002154 -0.000422  0.003556   
1990-12-02  0.002668  0.002177  0.004015  0.001815  0.012809  0.010528   
1990-12-09  0.005232  0.029573  0.012062 -0.001869  0.002823  0.005093   
1990-12-16  0.005154 -0.012387 -0.006591 -0.002224 -0.001695 -0.002407   
1990-12-23  0.003753  0.024725  0.008172  0.004691  0.002815  0

(Timestamp('2005-12-31 00:00:00', freq='A-DEC'),                   AA      AAPL        GE       IBM       JNJ      MSFT  \
2005-01-02  0.000102  0.001236 -0.001521  0.001752 -0.000568 -0.002176   
2005-01-09 -0.004579  0.015042 -0.002708 -0.005742 -0.002563 -0.000335   
2005-01-16 -0.005112  0.003624 -0.002654 -0.003521  0.000382 -0.004146   
2005-01-23 -0.008078  0.001024 -0.002648 -0.004558 -0.003427 -0.004536   
2005-01-30  0.000293  0.009765  0.003468  0.001120  0.008927  0.004114   
2005-02-06  0.003812  0.013026  0.002844  0.003488  0.004988  0.001136   
...              ...       ...       ...       ...       ...       ...   
2005-11-20 -0.002088  0.009783  0.006344  0.007533  0.006479  0.006316   
2005-11-27  0.008709  0.018077  0.003150  0.002922 -0.001548 -0.002780   
2005-12-04  0.005727  0.009646 -0.003886 -0.000341 -0.003026  0.001847   
2005-12-11  0.000358  0.004736  0.000228 -0.003779 -0.003663 -0.002153   
2005-12-18  0.000589 -0.008667  0.002940 -0.008364  0.002526 -0